# 一、處理樣本

## 1.處理訓練樣本

In [1]:
import pandas as pd
import numpy as np

d = pd.read_csv("cardQmerge.csv",encoding='utf-8')
# 剔除過長的樣本以避免 BERT 無法將整個輸入序列放入記憶體不多的 GPU
MAX_LENGTH = 350
d = d[~(d.text.apply(lambda x : len(x)) > MAX_LENGTH)] 

d = d.sample(frac=1, random_state=3345678) #(數量,亂數碼)
d = np.array(d)#np.ndarray()
d = d.tolist()
l_train = []
l_test = []
proportion = 0.8 #訓練比例，剩下為測試
for i in range(len(d)) :
    if i <= int(len(d)*proportion) :
        l_train.append(d[i])
    else :
        l_test.append(d[i])

df_train = pd.DataFrame(l_train)
df_test = pd.DataFrame(l_test)
df_train.columns = ['Q','text','s_position', 'e_position', 'A'] # 欄位重新命名
df_test.columns = ['Q','text','s_position', 'e_position', 'A'] # 欄位重新命名

df_train = df_train.loc[:, ['Q', 'text', 's_position', 'e_position']] # 只取需要的欄位
df_train.to_csv("card_train.tsv", sep="\t", index=False) # 轉成 tsv 檔減少讀取問題
print("預測樣本數：", len(df_train))
df_train.head(10)

預測樣本數： 649


,Q,text,s_position,e_position
0,"國內外,回饋金比例?",1.國內外一般消費回饋金1%，無最低門檻，回饋無上限,12,14
1,"台灣大車隊,刷卡金比例?",8.109/01/01~109/12/31搭乘台灣大車隊，嗶J卡悠遊卡功能扣款享10%刷卡金回饋,32,34
2,回饋類型?,1.109/01/01~109/12/31國內一般消費享1.5%現金積點回饋,24,28
3,現金回饋通路?,3.109/07/01~109/12/31以幣倍卡代繳指定16家壽險公司保費享1.2%現金回...,19,29
4,回饋類型?,1.109/01/01~109/12/31一般消費1%LINE POINTS回饋,19,21
5,回饋類型?,1.國內外一般消費回饋金1%，無最低門檻，回饋無上限,9,12
6,紅利通路?,1.109/01/01~109/12/31刷花旗紅利卡消費，單筆一般消費享每30元累積1點紅利,24,28
7,現金回饋通路?,1.109/01/01~109/12/31一般消費享0.5%現金回饋,13,17
8,"全台大眾交通,現金回饋比例?",6.109/01/01~109/12/31一卡通搭乘全台大眾交通，享2%現金回饋，回饋無上限,26,28
9,現金回饋通路?,2.109/01/01~109/12/31保險費1%現金回饋，現金回饋無上限｜以太陽卡刷卡繳...,13,16


## 2.初步處理測試樣本

In [2]:
# 去除不必要的欄位
df_test = df_test.loc[:, ['Q', 'text', 'A']] # 只取需要的欄位 A為原答案比較用 
df_test.to_csv("card_test.tsv", sep="\t", index=False) # 轉成 tsv 檔減少讀取問題

print("預測樣本數：", len(df_test))
df_test.head(10)

預測樣本數： 161


,Q,text,A
0,回饋類型?,1.109/01/01~109/12/31國內刷卡消費1%現金回饋，回饋無上限,現金回饋
1,回饋類型?,1.109/01/01~109/12/31，7大數位(網購/行動支付) : 最高6%現金回饋...,現金回饋
2,回饋類型?,2.109/01/01~109/9/30街口支付掃碼消費，非指定通路之一般消費，最高6%現金...,現金回饋
3,回饋類型?,1.每20元= 1點紅利點數,紅利點數
4,回饋類型?,3.109/01/01~109/12/31於快樂購特約商店消費100元可累計2點HAPPY ...,happygo點數
5,"一般消費,紅利比例?",4.109/01/01~109/12/31一般消費享15元=1點紅利回饋,15元=1點
6,"國外,現金回饋比例?",1.109/01/01~109/12/31國外一般消費，享不限金額一律現金回饋2%,2%
7,"一般消費,OPEN POINT比例?",1.109/07/01~109/12/31以玉山icash聯名卡刷卡消費享0.33%OPEN...,0.33%
8,LINE POINTS通路?,1.國內消費享LINE POINTS 2%回饋,國內
9,回饋類型?,2.109/01/01~109/12/31國內消費享每18元=1點飛行積金,飛行積金


In [3]:
# 樣本比統計
ratio = len(df_test) / len(df_train)
print("測試集樣本數 / 訓練集樣本數 = {:.1f} 倍".format(ratio))

測試集樣本數 / 訓練集樣本數 = 0.2 倍


## 3.將樣本處理成bert的輸入格式 (DataSet)

In [4]:
from torch.utils.data import Dataset
import torch
from transformers import BertTokenizer

class Dataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test"]  # 一般訓練你會需要 dev set
        self.mode = mode
        # 大數據你會需要用 iterator=True
        self.df = pd.read_csv("card_"+ mode + ".tsv", sep="\t").fillna("") # 讀檔轉成 pandas 格式
        self.len = len(self.df) #資料數量
        self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer
    
    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):  #測試樣本處理
        if self.mode == "test": # 沒標記 需產生結果時使用
            Q, text, A = self.df.iloc[idx,[0,1,2] ].values  # 取第 idx 筆資料，第 1、2、3 欄
            s_p = None
            e_p = None
        else: #訓練樣本處理
            Q, text, s_p, e_p = self.df.iloc[idx, :4].values  # 取第 idx 筆資料，第 1、2、3、4 欄
            A = None
        
        # 建立問句 Q 的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens_Q = self.tokenizer.tokenize(Q)
        word_pieces += tokens_Q + ["[SEP]"]
        len_Q = len(word_pieces) # 取 a 的長度
        
        # 文本 text 的 BERT tokens
        tokens_text = self.tokenizer.tokenize(text)
        word_pieces += tokens_text + ["[SEP]"]
        len_text = len(word_pieces) - len_Q # 取 b 的長度
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces) # 將單字串轉換成對映的 token 索引串
        tokens_tensor = torch.tensor(ids) # 建成向量式
        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        segments_tensor = torch.tensor([0] * len_Q + [1] * len_text, 
                                        dtype=torch.long)
              
        if self.mode != "test" : #問題長度加上才是整個輸入的位子
            s_p += len_Q
            e_p += len_Q
        
        return (tokens_tensor, segments_tensor, s_p, e_p)
    
    def __len__(self):
        return self.len
    
    
PRETRAINED_MODEL_NAME = "bert-base-multilingual-uncased"  # 指定繁簡中文 BERT-BASE 預訓練模型
# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞
trainset = Dataset("train", tokenizer=tokenizer)

In [5]:
# 選擇樣本
sample_idx = 1

# 將原始文本拿出做比較
Q, text, s_p, e_p = trainset.df.iloc[sample_idx].values #原始文本存在 trainset.df裡

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor, segments_tensor, s_p, e_p = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
問題  ：{Q}
文本  ：{text}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

segments_tensor：{segments_tensor}

s_p   ：{s_p}

e_p   ：{e_p}
--------------------

[還原 tokens_tensors]
{combined_text}
""")

[原始文本]
問題  ：台灣大車隊,刷卡金比例?
文本  ：8.109/01/01~109/12/31搭乘台灣大車隊，嗶J卡悠遊卡功能扣款享10%刷卡金回饋

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([  101,  2416,  5391,  2974,  8310,  9169,   117,  2190,  2338,  8688,
         4901,  1878,   136,   102,   129,   119, 16212,   120, 10454,   120,
        10454,   172, 16212,   120, 10189,   120, 10455,  4157,  1711,  2416,
         5391,  2974,  8310,  9169, 10032,  2656,   152,  2338,  3761,  8497,
         2338,  2236,  6990,  3945,  4835,  1754, 10148,   110,  2190,  2338,
         8688,  2735,  9457,   102])

segments_tensor：tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1])

s_p   ：46

e_p   ：48
--------------------

[還原 tokens_tensors]
[CLS]台灣大車隊,刷卡金比例?[SEP]8.109/01/01~109/12/31搭乘台灣大車隊，嗶j卡悠遊卡功能扣款享10%刷卡金回饋[SEP]



## 4.將 DataSet 輸入進 DataLoader 產生 mini-batch 使輸入可以批量處理 

In [6]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list，裡頭的每個 element 都是
# 剛剛定義的 `Dataset` 回傳的一個樣本，每個樣本都包含 4 tensors：
# - tokens_tensor
# - segments_tensor
# - s_p
# - e_p
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 測試集有 s_p, e_p
    if samples[0][2] is not None:
        s_p = ([s[2] for s in samples])
        s_p = torch.tensor(s_p)
        e_p = ([s[3] for s in samples])
        e_p = torch.tensor(e_p)
    else:
        s_p = None
        e_p = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, 
                                    batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, 
                                dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(
        tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, s_p, e_p


# 初始化一個每次回傳 8 個訓練樣本的 DataLoader
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
BATCH_SIZE = 8
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, 
                         collate_fn=create_mini_batch)

In [7]:
data = next(iter(trainloader))
tokens_tensors, segments_tensors, \
    masks_tensors, s_p, e_p = data

print("tokens_tensors.shape = ",{tokens_tensors.shape}) # 查看 tokens [資料數, 資料長度]
print(tokens_tensors)
print("------------------------")
print("segments_tensors = ",{segments_tensors.shape}) # #查看 Q,text標記[資料數, 資料長度]
print(segments_tensors)
print("------------------------")
print("masks_tensors = ",{masks_tensors.shape}) # 查看專注標記[資料數, 資料長度]
print(masks_tensors)
print("------------------------")
print("s_p = ",{s_p.shape}) # 查看答案起點[資料數]
print(s_p)
print("------------------------")
print("e_p = ",{e_p.shape}) #查看答案終點[資料數]
print(e_p)
print("------------------------")

tokens_tensors.shape =  {torch.Size([8, 151])}
tensor([[ 101, 2759, 2072,  ...,    0,    0,    0],
        [ 101, 2416, 5391,  ...,    0,    0,    0],
        [ 101, 2735, 9457,  ...,    0,    0,    0],
        ...,
        [ 101, 2735, 9457,  ...,    0,    0,    0],
        [ 101, 6553, 2183,  ...,    0,    0,    0],
        [ 101, 5721, 8688,  ...,    0,    0,    0]])
------------------------
segments_tensors =  {torch.Size([8, 151])}
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
------------------------
masks_tensors =  {torch.Size([8, 151])}
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
------------------------
s_p =  {torch.Size([8])}
tensor([

# 二、在 BERT 的基礎上加入 layer 建置新的下游任務模型

## 1.依任務的種類引入對映的模型 

### 新增一個 BERT 的預訓練模型 

In [8]:
#  載入模組
from transformers import BertForQuestionAnswering
from IPython.display import clear_output
import time #記錄用

PRETRAINED_MODEL_NAME = "bert-base-multilingual-uncased"

#輕量型
#model = BertForQuestionAnswering.from_pretrained(PRETRAINED_MODEL_NAME, num_hidden_layers = 8,
#                                                   num_attention_heads = 8)
    
#正常
model = BertForQuestionAnswering.from_pretrained(PRETRAINED_MODEL_NAME)

clear_output()

# high-level 顯示此模型裡的 modules
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
qa_outputs      Linear(in_features=768, out_features=2, bias=True)


### 或載入以訓練模型(沒有要重新訓練 BERT 時用)

In [ ]:
from transformers import BertForQuestionAnswering
from IPython.display import clear_output
import time #記錄用

#del model #測試用 刪除原 model
PRETRAINED_MODEL_NAME = "bert-base-multilingual-uncased"
path = "model\\Nx6\\" + "pytorch_model.bin"

model = BertForQuestionAnswering.from_pretrained(PRETRAINED_MODEL_NAME)
model.load_state_dict(torch.load(path))

print(model) #測試用

## 2.使用還沒訓練的模型對樣本做比對確認模型可使用

In [9]:
def get_predictions(model, dataloader, compute_acc=False):
    predictionSP = None
    predictionEP = None
    corrects = 0
    correcte = 0
    correct = 0
    total = 0
    PA = []
    c = 1
    
    with torch.no_grad():
        # 遍巡整個資料集
        print(time.strftime("%Y%m%d_%H:%M:%S", time.localtime()) + ' 比對開始 總共' + str(len(dataloader)) + '筆') #總比數
        for data in dataloader:
            if (c%int((len(dataloader)/5))==1 or c==len(dataloader)) :#每20%顯示一次
                print(time.strftime("%Y%m%d_%H:%M:%S", time.localtime())+' 比對進度：' + str(c) + '/' + str(len(dataloader)))
            c+=1
            
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda: #沒 GPU 啟動不了QQ
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors) # 沒 label 的狀況下 [0] 是兩陣列紀錄各 token 預測答案開始與結束L位子分數
           
            logit_s = outputs[0] # 取得各 token 預測答案開始位子分數
            logit_e = outputs[1] # 取得各 token 預測答案結束位子分數
            
            _, predSP = torch.max(logit_s.data, 1) # 依最高分的取得預測答案開始位子
            _, predEP = torch.max(logit_e.data, 1) # 依最高分的取得預測答案結束位子
            # 用來計算訓練集的分類準確率
            if compute_acc:
                s_p = data[3]
                e_p = data[4]
                total += s_p.size(0)
                for i in range (len(predSP)) :
                    if((predSP[i] == s_p[i]) and (predEP[i] == e_p[i])) : # 開始與結束的預測都與實際相同
                        correct += 1
                    if(predSP[i] == s_p[i]) : # 開始的預測與實際相同
                        corrects += 1
                    if(predEP[i] == e_p[i]) : # 結束的預測與實際相同
                        correcte += 1
            else : #不是訓練模式從 data 產生預測答案
                SP = predSP.tolist()
                EP = predEP.tolist()
                for i in range(len(data[0])):
                    all_tokens = tokenizer.convert_ids_to_tokens(tokens_tensors[i].tolist()) # 將 tensor token 轉成 list 對映文字
                    PA.append("".join(all_tokens[ SP[i] : EP[i]])) # 將答案預測答案存成 list
                
            # 將當前 batch 記錄下來
            if predictionSP is None:
                predictionSP = predSP
                predictionEP = predEP
            else:
                predictionSP= torch.cat((predictionSP,predSP))
                predictionEP= torch.cat((predictionEP,predEP))
    
    if compute_acc:
        acc = correct / total
        accs = corrects / total
        acce = correcte / total
        return predictionSP, predictionEP, accs, acce, acc
    return PA
    
# 讓模型跑在 GPU 上並取得訓練集的準確率
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu") # 強制使用 CPU
print("device:", device)
model = model.to(device)
_, _, accs, acce, acc = get_predictions(model, trainloader, compute_acc=True)
print("acc:", acc, "accs:", accs, "acce:", acce)

device: cpu
20200828_09:51:18 比對開始 總共82筆
20200828_09:51:18 比對進度：1/82
20200828_09:51:45 比對進度：17/82
20200828_09:52:12 比對進度：33/82
20200828_09:52:37 比對進度：49/82
20200828_09:53:08 比對進度：65/82
20200828_09:53:35 比對進度：81/82
20200828_09:53:37 比對進度：82/82
acc: 0.0 accs: 0.0061633281972265025 acce: 0.0015408320493066256


### 模型儲存函數

In [10]:
import time
import os
def model_save(model) :
    path = "model\\" + time.strftime("%Y%m%d_%H%M%S", time.localtime())
    if not os.path.isdir(path): # 偵測有無資料夾
        os.mkdir(path) # 開啟新資料夾
    model.save_pretrained(path)

## 3.訓練模型並比對

In [11]:
%%time

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

print(time.strftime("%Y%m%d_%H:%M:%S", time.localtime()) + ' 訓練開始 總共' + str(len(trainloader)) + '筆') #總比數
EPOCHS = 6  # 重複訓練次數 依你的 mini-batch
for epoch in range(EPOCHS):
    
    print('--------------------------------------------------------------------')
    print(time.strftime("%Y%m%d_%H:%M:%S", time.localtime()) +' 訓練第' + str(epoch+1) + '次')
    c=1
    running_loss = 0.0
    for data in trainloader:
        
        if (c%int((len(trainloader)/5))==1 or c==len(trainloader)) :#每20%顯示一次
            print(time.strftime("%Y%m%d_%H:%M:%S", time.localtime()) +' 訓練進度：' + str(c) + '/' + str(len(trainloader)))
        c+=1
        
        tokens_tensors, segments_tensors, \
        masks_tensors, s_p, e_p = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        start_positions=s_p,
                        end_positions=e_p) # 有 start_positions, end_positions 的狀況下 [0] 是 loss 

        loss = outputs[0] # loss
        # backward
        loss.backward()
        optimizer.step()
        
        
        # 紀錄當前 batch loss
        running_loss += loss.item()
    print('--------------------------------------------------------------------')
    # 計算分類準確率
    _, _, accs, acce, acc = get_predictions(model, trainloader, compute_acc=True)

    print('[epoch %d] loss: %.2f, acc: %.2f,' %(epoch + 1, running_loss, acc ))
    print('           accs: %.2f, acce: %.2f' % (accs, acce))
    model_save(model) #自動儲存

20200828_09:53:37 訓練開始 總共82筆
--------------------------------------------------------------------
20200828_09:53:37 訓練第1次
20200828_09:53:37 訓練進度：1/82
20200828_09:55:20 訓練進度：17/82
20200828_09:57:03 訓練進度：33/82
20200828_09:58:33 訓練進度：49/82
20200828_10:00:12 訓練進度：65/82
20200828_10:01:42 訓練進度：81/82
20200828_10:01:48 訓練進度：82/82
--------------------------------------------------------------------
20200828_10:01:49 比對開始 總共82筆
20200828_10:01:49 比對進度：1/82
20200828_10:02:22 比對進度：17/82
20200828_10:02:54 比對進度：33/82
20200828_10:03:23 比對進度：49/82
20200828_10:03:56 比對進度：65/82
20200828_10:04:25 比對進度：81/82
20200828_10:04:27 比對進度：82/82
[epoch 1] loss: 219.03, acc: 0.40,
           accs: 0.71, acce: 0.49
--------------------------------------------------------------------
20200828_10:04:30 訓練第2次
20200828_10:04:30 訓練進度：1/82
20200828_10:06:08 訓練進度：17/82
20200828_10:07:48 訓練進度：33/82
20200828_10:09:19 訓練進度：49/82
20200828_10:10:55 訓練進度：65/82
20200828_10:12:24 訓練進度：81/82
20200828_10:12:29 訓練進度：82/82
------------

# 三、產生測試資料的預測答案 ，輸出成 CSV 格式

## 產生含原答案，跟預測答案的 CSV 檔

In [12]:
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size
testset = Dataset("test", tokenizer=tokenizer)
testloader = DataLoader(testset, batch_size=4, 
                        collate_fn=create_mini_batch)

# 用分類模型預測測試集
predictions = get_predictions(model, testloader)

df = pd.DataFrame({"P_A": predictions})
df_pred = pd.concat([testset.df.loc[:, ["Q","text","A"]], 
                          df.loc[:, 'P_A']], axis=1)
df_pred.to_csv('bert_prec_cardQmerge_80Nx6.csv', index=False)
df_pred.head(20)

20200828_10:57:27 比對開始 總共41筆
20200828_10:57:27 比對進度：1/41
20200828_10:57:34 比對進度：9/41
20200828_10:57:40 比對進度：17/41
20200828_10:57:46 比對進度：25/41
20200828_10:57:51 比對進度：33/41
20200828_10:57:58 比對進度：41/41


,Q,text,A,P_A
0,回饋類型?,1.109/01/01~109/12/31國內刷卡消費1%現金回饋，回饋無上限,現金回饋,現金回饋
1,回饋類型?,1.109/01/01~109/12/31，7大數位(網購/行動支付) : 最高6%現金回饋...,現金回饋,現金回饋
2,回饋類型?,2.109/01/01~109/9/30街口支付掃碼消費，非指定通路之一般消費，最高6%現金...,現金回饋,現金回饋
3,回饋類型?,1.每20元= 1點紅利點數,紅利點數,紅利點數
4,回饋類型?,3.109/01/01~109/12/31於快樂購特約商店消費100元可累計2點HAPPY ...,happygo點數,happygo點數
5,"一般消費,紅利比例?",4.109/01/01~109/12/31一般消費享15元=1點紅利回饋,15元=1點,15元=1點
6,"國外,現金回饋比例?",1.109/01/01~109/12/31國外一般消費，享不限金額一律現金回饋2%,2%,2%
7,"一般消費,OPEN POINT比例?",1.109/07/01~109/12/31以玉山icash聯名卡刷卡消費享0.33%OPEN...,0.33%,0.33%
8,LINE POINTS通路?,1.國內消費享LINE POINTS 2%回饋,國內,國內
9,回饋類型?,2.109/01/01~109/12/31國內消費享每18元=1點飛行積金,飛行積金,飛行積金


In [14]:
import pandas as pd

df_pred = pd.read_csv("bert_prec_cardQmerge_80Nx6.csv",header=None,index_col=None)

correct = 0
total = len(df_pred[0])
nanCount = 0
for i in range(total) :
    if(df_pred[2][i] == df_pred[3][i]) :
        
        correct+=1
    else :
        print(i,"|",df_pred[0][i],"|",df_pred[1][i],"|",df_pred[2][i],"|",df_pred[3][i])
print("------------------------------")
print("total answer：",total-1," as same：",correct)
print("acc:%.4f"% (correct/(total-1)))

0 | Q | text | A | P_A
12 | OPEN POINT通路? | 3.持本卡至全台7-ELEVEN門市以icash功能消費，單筆交易全額由icash功能支付，且以發票上以icash功能實際支付的消費金額為準，每消費1元享0.33% OPEN POINT點數回饋 | 全台7-eleven門市 | 7-eleven門市
13 | 現金回饋通路? | 3.109/01/01~109/12/31中日韓一般消費享3%現金回饋 | 中日韓 | nan
14 | 紅利點數通路? | 2.109/01/01~109/12/31除指定通路現金回饋其他一般消費可享20元=1點紅利點數 | 一般消費 | nan
17 | 回饋類型? | 3.109/01/01~109/12/31刷保費依下列門檻採級距回饋｜未滿2萬：紅利回饋20元=1點｜2萬以上~未滿20萬：1%現金回饋｜20萬以上~未滿50萬：1.2%現金回饋｜50萬以上：1.5%現金回饋 | 現金回饋 | nan
21 | 現金回饋通路? | 2.109/01/01~109/12/31當期信用卡帳單一般消費5,000(含)元以上，於當期之指定網購/超商/交通/餐廳通路消費，享5%現金回饋｜*指定通路附上連結，https://cathaybk.com.tw/cathaybk/personal/credit-card/cards/intro/koko-combo-credit-icash/#tab-02  | 指定網購/超商/交通/餐廳通路 | 信用卡帳單一般消費5,000(含)元以上，於當期之指定網購/超商/交通/餐廳通路
23 | 回饋類型? | 3.109/01/01~109/12/31刷保費依下列門檻採級距回饋｜未滿2萬：紅利回饋20元=1點｜2萬以上~未滿20萬：1%現金回饋｜20萬以上~未滿50萬：1.2%現金回饋｜50萬以上：1.5%現金回饋 | 現金回饋 | nan
26 | 回饋類型? | 3.持本卡至全台7-ELEVEN門市以icash功能消費，單筆交易全額由icash功能支付，且以發票上以icash功能實際支付的消費金額為準，每消費1元享0.33% OPEN POINT點數回饋 | openpoint | openpoint點數
31 | 現金回饋通路? | 3.109/05/15~109/0